In [1]:
from keras import backend as K
import tensorflow as tf

from SSD_generate_anchors import generate_ssd_priors
from CustomDataLoader import DataAugmentor
from utility import *
from pathlib import Path
import collections
from ssd300_model import SSD300
from skimage.io import imread, imshow
import cv2

import numpy as np
from ssd_loss_function_v2 import CustomLoss





Using TensorFlow backend.


In [2]:
# root = Path.home()/'data'/'VOCdevkit'/'VOC2007'
root  = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'/'VOC2007'
voc_2007_datafile = root/'ImageSets'/'Main'/'train.txt'
voc_2007_images   = root/'JPEGImages'
voc_2007_annotations = root/'Annotations'



In [3]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]


In [4]:
priors = generate_ssd_priors(specs)
num_priors = priors.shape[0]

In [5]:
batch_size = 10

testloader = DataAugmentor()
data       = testloader.flow_from_directory(root = root,
                                            data_file=voc_2007_datafile,
                                            target_size=300,
                                            batch_size = batch_size,
                                            shuffle    = True,
                                            num_classes = 21,
                                            priors      = priors
                                            )

sample = data[0]

images, targets = sample

In [6]:
print(images.shape)
print(targets.shape)

(10, 300, 300, 3)
(10, 8732, 5)


In [7]:
model = SSD300(input_shape=(300,300, 3), num_classes=21)

prediction = model.predict(images)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
SSDLoss = CustomLoss(anchors   = priors, 
                     alpha     = 1.0
                    )


In [9]:
conf_t, conf_p = SSDLoss(y_true = targets, y_pred = prediction)


In [10]:
print(conf_t.shape)
print(conf_p.shape)

(10, 8732)
(10, 8732, 21)


In [11]:
batch_conf = conf_p.reshape(-1, 21)
index      = conf_t.reshape(-1, 1).astype('int')
print(batch_conf.shape)
print(index.shape)

(87320, 21)
(87320, 1)


In [12]:
loss_c = log_sum_exp(batch_conf) - gather(batch_conf, dim=1, index=index )

In [13]:
print(loss_c)
print(loss_c.shape)

[[3.0439827]
 [3.054411 ]
 [3.0523968]
 ...
 [3.12317  ]
 [3.0445225]
 [3.0445225]]
(87320, 1)


In [ ]:
batch_conf_ = K.reshape(conf_p, shape = (-1, 21))
index_      = K.reshape(conf_t, shape=(-1,1))
batch_conf_ = K.cast(batch_conf, 'float32')
index_      = K.cast(index_, 'int32')

loss_c_ = K.gather(conf_p, index_)


(87320, 1, 8732, 21)


ResourceExhaustedError: OOM when allocating tensor with shape[87320,1,8732,21] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node embedding_lookup_3 (defined at /home/vlsilab/anaconda2/envs/py3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1228) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_lookup_3', defined at:
  File "/home/vlsilab/anaconda2/envs/py3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/vlsilab/anaconda2/envs/py3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/vlsilab/anaconda2/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/vlsilab/anaconda2/envs/py3/lib/python3.7/site-packages/traitlets/config/application.py",